# Lateral Raise Detection and Counter
This notebook uses Mediapipe and OpenCV to detect, validate, and count repetitions of lateral raises. It ensures proper form while visualizing angles and exercise stages.

### References and Setup
- Mediapipe Documentation: [Mediapipe Solutions Guide](https://ai.google.dev/edge/mediapipe/solutions/guide)
- This notebook relies on the Mediapipe library for pose estimation and OpenCV for video input processing.

In [ ]:
# Install and import necessary libraries
%pip install mediapipe opencv-python
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils


### Pose Detection and Analysis
- Setup Mediapipe's Pose module to detect body landmarks.
- Calculate angles between key landmarks to assess lateral raises.

In [ ]:
# Function to calculate angles between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point
    c = np.array(c)  # End point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360.0 - angle
    
    return angle


### Exercise Validation and Counting
- Validate proper form for lateral raises.
- Track repetitions by detecting movement stages (`up` and `down`).

In [ ]:
# Initialize counters and stage
counter = 0
stage = None


### Visualization
- Draw landmarks and pose connections using Mediapipe utilities.
- Display angles and movement stages (`up/down`) visually on the video feed.

In [ ]:
# Function to visualize angles and stages
def visualize(image, left_angle, right_angle, left_elbow, right_elbow, stage):
    # Display angles on the video feed
    cv2.putText(image, f'Left: {int(left_angle)}', 
                tuple(np.multiply(left_elbow, [640, 480]).astype(int)), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(image, f'Right: {int(right_angle)}', 
                tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    
    # Display exercise stage
    cv2.putText(image, f'Stage: {stage}', (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2, cv2.LINE_AA)


### Execution Environment
- A loop to process real-time video input for dynamic interaction.
- Detects poses, calculates angles, validates form, counts repetitions, and visualizes the output.

In [ ]:
# Open video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    
    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # Make detections
    results = pose.process(image)
    
    # Recolor image back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Extract landmarks
    try:
        landmarks = results.pose_landmarks.landmark
        
        # Get coordinates for lateral raises (both arms)
        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        
        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        
        # Calculate angles
        left_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        right_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        
        # Lateral raise logic
        if left_angle > 80 and right_angle > 80:
            stage = "up"
        if left_angle < 30 and right_angle < 30 and stage == "up":
            stage = "down"
            counter += 1
            print(counter)
        
        # Visualize output
        visualize(image, left_angle, right_angle, left_elbow, right_elbow, stage)
        
    except:
        pass
    
    # Render detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    # Display video feed
    cv2.imshow('Lateral Raises', image)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
